In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping

c:\Users\abhinav bhardwaj\miniconda3\envs\ml\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.0)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


In [2]:
train_data_dir = 'dataset'

In [3]:

train_datagen = ImageDataGenerator(
    rescale=1./255.,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)


train_generator = train_datagen.flow_from_directory(
    train_data_dir, 
    batch_size=32,
    class_mode='binary', 
    target_size=(224, 224)
)


Found 2829 images belonging to 2 classes.


In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [5]:
base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

for layer in base_model.layers[:100]:
    layer.trainable = False
for layer in base_model.layers[100:]:
    layer.trainable = True

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

early_stopping = EarlyStopping(monitor='acc', patience=5, restore_best_weights=True)

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss='binary_crossentropy', 
              metrics=['acc'])
inception_hist = model.fit(train_generator, 
                           steps_per_epoch=len(train_generator),
                           epochs=15,
                           callbacks=[early_stopping])

print("Final Accuracy: ", inception_hist.history['acc'][-1])

c:\Users\abhinav bhardwaj\miniconda3\envs\ml\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/15


UnimplementedError: Graph execution error:

Detected at node 'model/conv2d/Conv2D' defined at (most recent call last):
    File "c:\Users\abhinav bhardwaj\miniconda3\envs\ml\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\abhinav bhardwaj\miniconda3\envs\ml\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\abhinav bhardwaj\miniconda3\envs\ml\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\abhinav bhardwaj\miniconda3\envs\ml\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\abhinav bhardwaj\miniconda3\envs\ml\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\abhinav bhardwaj\miniconda3\envs\ml\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever(